In [0]:
!pip install transformers

     |████████████████████████████████| 501kB 7.2MB/s 
     |████████████████████████████████| 3.7MB 16.2MB/s 
     |████████████████████████████████| 1.0MB 47.6MB/s 
     |████████████████████████████████| 870kB 48.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=c4c97a1ccb61f2fdba714419773bc4ec578e005a95fa6effbcbe6e0c0da12e5e
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
#Also based on the following tutorials
#https://mccormickml.com/2019/07/22/BERT-fine-tuning/

In [0]:
import tensorflow as tf
import torch

import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


from transformers import BertTokenizer as bertTokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler

from sklearn.svm import SVC
from sklearn import preprocessing

from transformers import BertForSequenceClassification as bfsc,AdamW,BertConfig
from torch.utils.data import Dataset
from transformers import get_linear_schedule_with_warmup


from __future__ import absolute_import, division, print_function

import glob
import logging
import os
import random
import json

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import random
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm_notebook, trange

from transformers import (WEIGHTS_NAME, BertConfig, BertForSequenceClassification, BertTokenizer,
                                  XLMConfig, XLMForSequenceClassification, XLMTokenizer, 
                                  XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer,
                                  RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)

from transformers import AdamW#, WarmupLinearSchedule

#from utils import (convert_examples_to_features,
                        #output_modes, processors)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [0]:
gpuname=""
device=""
y=""
preprocessedTweets=""
ids_of_sentence=[]
ids_of_sentence_words=[]
attention_masks=[]

In [0]:


gpuname=tf.test.gpu_device_name()
if gpuname=='/device:GPU:0':
  print('Found GPU at :{}'.format(gpuname))
else:
  gpuname=""
if torch.cuda.is_available():
  device=torch.device("cuda")
  n_gpu=torch.cuda.device_count()
  print("The device name is %s"%torch.cuda.get_device_name(0))
else:
  print("No GPU available using only CPU instead")
  device=torch.device("cpu")




Found GPU at :/device:GPU:0
The device name is Tesla K80


In [0]:
MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
    'xlnet': (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer),
    'xlm': (XLMConfig, XLMForSequenceClassification, XLMTokenizer),
    'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
}

#config_class, model_class, tokenizer_class = MODEL_CLASSES[args['model_type']]
config_class, model_class, tokenizer_class = MODEL_CLASSES['roberta']


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
##NOT FOR SYSTEMS
!unzip -P ****** -qq '/content/drive/My Drive/EnglishData/task_a_distant.zip' 

In [0]:

#GET THE DATA FROM THE PANDAS FRAME
headers=['id','tweet','subtask_a','subtask_b','subtask_c']
englishdata = pd.read_csv("olid-training-v1.0.tsv", delimiter='\t',names=headers,low_memory=False)
englishdata=englishdata[['id','tweet','subtask_a']]

In [0]:
'''# conversion to float taken from https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options
def convertToFloat(val):
    if not val:
        return 0    
    try:
        return np.float64(val)
    except:        
        return np.float64(0)


#GET THE DATA FROM THE PANDAS FRAME
headers=['id','text','average','std']
englishdata = pd.read_csv("task_a_distant.tsv", delimiter='\t',names=headers,low_memory=False,converters={"average":convertToFloat,"std":convertToFloat})
englishdata=englishdata[:100]
'''

In [0]:
len(englishdata)

13241

In [0]:
englishdata.head()

,id,tweet,subtask_a
0,id,tweet,subtask_a
1,86426,@USER She should ask a few native Americans wh...,OFF
2,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
3,16820,Amazon is investigating Chinese employees who ...,NOT
4,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF


In [0]:
englishtrain,englishtest= train_test_split(englishdata, test_size=0.2, random_state=42)
export_csv = englishtrain.to_csv ('/content/drive/My Drive/EnglishData/olidlearn/TrainFileEnglish.tsv', index = None, header=True)
print (englishtrain.head())
#englishtest,englishpredict= train_test_split(englishtemp, test_size=0.5, random_state=42)
export_csv = englishtest.to_csv ('/content/drive/My Drive/EnglishData/olidlearn/TestFileEnglish.tsv', index = None, header=True)
print (englishtest.head())
#export_csv = englishpredict.to_csv ('/content/drive/My Drive/EnglishData/predictFileEnglish.csv', index = None, header=True)
#print (englishpredict.head())

         id                                              tweet subtask_a
7945  95036                 @USER @USER Mehn you are too sweet       NOT
9185  98619  @USER Not if you are polling mexicans!!! #taco...       NOT
1056  96330  @USER @USER then you are not equipped to adopt...       NOT
1146  33064  @USER @USER @USER We've had a good conversatio...       NOT
3827  20123  @USER @USER We all know this is another Libera...       OFF
          id                                              tweet subtask_a
12388  27650  @USER @USER Why is John Kerry running his mout...       NOT
385    53995  @USER We shouldn’t be surprised. These same pi...       OFF
4896   72139  @USER You are so right on this issue and Democ...       OFF
3971   95266  @USER @USER But it’s so cute when conservative...       NOT
11895  32891  @USER @USER @USER @USER @USER @USER @USER @USE...       NOT


In [0]:
ids_of_sentence=[]
ids_of_sentence_words=[]
attention_masks=[]

def giveIds(sentence):
  maxlength=0
  tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
  for t in sentence:
      tokenized_sentence_id=tokenizer.encode(t,add_special_tokens=True)
      if(maxlength<len(tokenized_sentence_id)):
          maxlength=len(tokenized_sentence_id)
  return maxlength

In [0]:
max_length=

In [0]:

# conversion to float taken from https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options
def convertToFloat(val):
    if not val:
        return 0    
    try:
        return np.float64(val)
    except:        
        return np.float64(0)

def giveLabel(y):
  i=0
  y1=[]
  for r in y:
    if(y[i]>=0.5):
      y1.append(1)
    else:
      y1.append(0)
    i=i+1;
  return y1

def readDataTrain():
  headers=['id','text','average','std']
  edata = pd.read_csv("/content/drive/My Drive/EnglishData/TrainFileEnglish.csv", delimiter=',',names=headers,low_memory=False,converters={"average":convertToFloat,"std":convertToFloat})
  edata=edata[1:]
  dfnumpy=edata.to_numpy();
  X=dfnumpy[:, 1].reshape(-1, 1)
  y=dfnumpy[:, 2].reshape(-1, 1)
  y1=giveLabel(y)
  return X,y1

def readDataTest():
  headers=['id','text','average','std']
  edata = pd.read_csv("/content/drive/My Drive/EnglishData/TestFileEnglish.csv", delimiter=',',names=headers,low_memory=False,converters={"average":convertToFloat,"std":convertToFloat})
  edata=edata[1:]
  dfnumpy=edata.to_numpy();
  X=dfnumpy[:, 1].reshape(-1, 1)
  y=dfnumpy[:, 2].reshape(-1, 1)
  y1=giveLabel(y)
  return X,y1

In [0]:
x_train,y_train=readDataTrain()
x_test,y_test=readDataTrain()


In [0]:
###IMPORTANT
'''xtest,x_test_mask=giveIds(x_test.flatten(),y_test)
x_test_pytorch=torch.tensor(xtest)
y_test_pytorch=torch.tensor(y_test)
x_test_mask_pytorch=torch.tensor(x_test_mask)
tedata=TensorDataset(x_test_pytorch,x_test_mask_pytorch,y_test_pytorch)
tesampler=RandomSampler(tedata)
bsize=64
tedataloader=DataLoader(tedata,sampler=tesampler,batch_size=bsize)
print(len(xtest))
print(len(y_test))
len(x_train)
'''

'xtest,x_test_mask=giveIds(x_test.flatten(),y_test)\nx_test_pytorch=torch.tensor(xtest)\ny_test_pytorch=torch.tensor(y_test)\nx_test_mask_pytorch=torch.tensor(x_test_mask)\ntedata=TensorDataset(x_test_pytorch,x_test_mask_pytorch,y_test_pytorch)\ntesampler=RandomSampler(tedata)\nbsize=64\ntedataloader=DataLoader(tedata,sampler=tesampler,batch_size=bsize)\nprint(len(xtest))\nprint(len(y_test))\nlen(x_train)\n'

In [0]:
from transformers import BertForSequenceClassification as bfsc,AdamW,BertConfig
model=bfsc.from_pretrained('bert-base-uncased',num_labels=2,output_attentions=False,output_hidden_states=False)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
params=list(model.named_parameters())

In [0]:
no_decay = ["bias", "beta","LayerNorm.weight","gamma"]
optimizer_grouped_parameters = [
{
"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
"weight_decay": 0.01,
},
{"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]

In [0]:
optimizer=AdamW(model.parameters(),lr=2e-5,eps=1e-8)

In [0]:

torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/EnglishData/bertenglish.pth.tar')
checkpoint = torch.load('/content/drive/My Drive/EnglishData/bertenglish.pth.tar')
model.load_state_dict(checkpoint['state_dict'])
  

<All keys matched successfully>

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def calculateF1Score(predictions,labels):
  #rowwise return the index of the max element ie 0 or 1 depending on the maximum value returned
  predictionArgmax=np.argmax(predictions,axis=1).flatten()
  labelsFlattend=labels.flatten()
  print("Predictions Argmax",predictionArgmax)
  print("labels Flattened",labelsFlattend)   
  return f1_score(labelsFlattend, predictionArgmax, average='macro'),accuracy_score(labelsFlattend, predictionArgmax)



In [0]:
###IMPORTANT
'''
xtest,x_test_mask=giveIds(x_test.flatten(),y_test)
x_test_pytorch=torch.tensor(xtest)
y_test_pytorch=torch.tensor(y_test)
x_test_mask_pytorch=torch.tensor(x_test_mask)
tedata=TensorDataset(x_test_pytorch,x_test_mask_pytorch,y_test_pytorch)
tesampler=RandomSampler(tedata)
bsize=64
tedataloader=DataLoader(tedata,sampler=tesampler,batch_size=bsize)
print(len(xtest))
print(len(y_test))
len(x_train)
'''

'\nxtest,x_test_mask=giveIds(x_test.flatten(),y_test)\nx_test_pytorch=torch.tensor(xtest)\ny_test_pytorch=torch.tensor(y_test)\nx_test_mask_pytorch=torch.tensor(x_test_mask)\ntedata=TensorDataset(x_test_pytorch,x_test_mask_pytorch,y_test_pytorch)\ntesampler=RandomSampler(tedata)\nbsize=64\ntedataloader=DataLoader(tedata,sampler=tesampler,batch_size=bsize)\nprint(len(xtest))\nprint(len(y_test))\nlen(x_train)\n'

In [0]:
#z=len(x_test)

In [0]:
#creating a dataset inspired from 
#https://towardsdatascience.com/bert-classifier-just-another-pytorch-model-881b3cf05784





In [0]:
#xytest[1]

In [0]:
#the customdataset section has been inspired from 
#https://github.com/sugi-chan/custom_bert_pipeline/blob/master/bert_pipeline.ipynb  

In [0]:
class EnglishTrainDataset(Dataset):
    def __init__(self,xytrain):
        self.xytrain = xytrain
        self.maxlength=MAXLENGTH
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytrain[0][index].flatten()))
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_train_pytorch = torch.tensor(ids_of_sentence_word)
        y_train_pytorch=torch.tensor(self.xytrain[1][index])
        x_train_mask_pytorch=torch.tensor(attention_mask)
        
        return x_train_pytorch,x_train_mask_pytorch,y_train_pytorch
        #return [1,2,3]
        
    def __len__(self):
        return len(self.xytrain[0])
 

'''
z=0;
for batch_idx, data in enumerate(tdataloader): 
  if z==100:
    break;
  z=z+1;'''

'\nz=0;\nfor batch_idx, data in enumerate(tdataloader): \n  if z==100:\n    break;\n  z=z+1;'

In [0]:
class EnglishTestDataset(Dataset):
    def __init__(self,xytest):
        self.xytest = xytest
        self.maxlength=MAXLENGTH
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytest[0][index].flatten()))
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_test_pytorch = torch.tensor(ids_of_sentence_word)
        y_test_pytorch=torch.tensor(self.xytest[1][index])
        x_test_mask_pytorch=torch.tensor(attention_mask)
        
        return x_test_pytorch,x_test_mask_pytorch,y_test_pytorch
        #return [1,2,3]
        
    def __len__(self):
        return len(self.xytest[0])



In [0]:
#tokenized_review = tokenizer.tokenize(str(xytest[0][0].flatten()))
 #y_test_pytorch=torch.tensor(y_test[0])
 #y_test[0]

In [0]:
#tokenized_review

In [0]:
englishdata=""
englishtest=""
englishtrain=""
X=""
y=""

In [0]:
import random
import time 

def set_seed(seed,ngpu):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if ngpu > 0:
        torch.cuda.manual_seed_all(seed)
      
set_seed(42,torch.cuda.device_count())
#remove later

def trainData(tdataloader,tedataloader):
  epochs=4
  lossList=[]
  max_grad_norm=1.0
  for e in range(0, epochs):
      print("Start Epoch Number",(e + 1))
      print("Start Training")
      if device.type=="cpu":
       model.to(device)
       map_location='cpu'
      else:
        model.cuda()
        map_location=lambda storage, loc: storage.cuda()
      checkpoint = torch.load('/content/drive/My Drive/EnglishData/bertenglish.pth.tar',map_location=map_location)
      model.load_state_dict(checkpoint['state_dict'])
  
      #Amount of time taken for training
      t1 = time.time()
      tr_loss, logging_loss = 0.0, 0.0
      model.train()
      tsteps=0
      for step, batch in enumerate(tdataloader):
          if step % 50 == 0 and not step == 0:
              print("Batch Completed  {:,}  of  {:,}.    Elapsed time is  {}".format(step, len(tdataloader),time.time() - t1))
          batch = tuple(t.to(device) for t in batch)
          inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
          model.zero_grad()        
          outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"], labels=inputs["labels"])
          loss = outputs[0]
          loss.backward()
          tr_loss += loss.item()
          torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
          tsteps+=1
          optimizer.step()
          sch.step()
      a_tr_loss = tr_loss /(tsteps)               
      lossList.append(a_tr_loss)
      print(" The training loss incured is  {0:.3f}".format(a_tr_loss))
      t2=time.time()
      print("  Training one epoch time taken",t2-t1)
      print(" Validation starts here ")
      t1 = time.time()
      model.eval()
      eval_loss = 0
      nb_eval_steps = 0
      eval_f1=0
      eval_acc=0
      
      for batch_idx, data in enumerate(tedataloader):
          
          batch = tuple(t.to(device) for t in data)            
          inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
          with torch.no_grad():        
             outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"])
          logits = outputs[0]
          logits = logits.detach().cpu().numpy()
          label_ids = (inputs["labels"]).to('cpu').numpy()
          tmpf1score,tmpaccscore = calculateF1Score(logits, label_ids)
          eval_f1 = eval_f1+tmpf1score
          eval_acc=eval_acc+tmpaccscore
          nb_eval_steps += 1
          #print(" TEMP F1 score: {0:.3f}".format(tmpf1score))
          #print("TEMP  Accuracy score: {0:.3f}".format(tmpaccscore))
      torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/EnglishData/bertenglish.pth.tar')

      print("  F1 score: {0:.3f}".format(eval_f1/nb_eval_steps))
      print("  Accuracy score: {0:.3f}".format(eval_acc/nb_eval_steps))
      t2=time.time()
      print("  Validating one epoch time taken ",t2-t1)
      
    
  print("ALL DONE!!!")

In [0]:
#xytrain=[x_train[:8000],y_train[:8000]]\
MAXLENGTH=giveIds(x_train[0])
xytrain=[x_train,y_train]
tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
tdataset = EnglishTrainDataset(xytrain)
tsampler=RandomSampler(tdataset)
tdataloader = DataLoader(tdataset, batch_size=32, num_workers=1, shuffle=False,sampler=tsampler)
      
epochs=4
total_steps=len(tdataloader)*epochs
sch=get_linear_schedule_with_warmup(optimizer,
                                    num_warmup_steps=0,num_training_steps=total_steps)

#xytest=[x_test[:8000],y_test[:8000]]
xytest=[x_test,y_test]
tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
tedataset = EnglishTestDataset(xytest) 
tesampler=RandomSampler(tedataset)
tedataloader = DataLoader(tedataset, batch_size=32, num_workers=1, shuffle=False,sampler=tesampler)
trainData(tdataloader,tedataloader)



Start Epoch Number 1
Start Training
 The training loss incured is  0.661
  Training one epoch time taken 0.7199721336364746
 Validation starts here 
Predictions Argmax [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labels Flattened [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
Predictions Argmax [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labels Flattened [1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0]
Predictions Argmax [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labels Flattened [1 1 1 0 0 0 0 0 0 1 1 1 0 1 0 0]
  F1 score: 0.421
  Accuracy score: 0.740
  Validating one epoch time taken  1.3960075378417969
Start Epoch Number 2
Start Training
 The training loss incured is  0.537
  Training one epoch time taken 0.4642632007598877
 Validation starts here 
Predictions Argmax [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labels Flattened [0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 

In [0]:
'''nlists=2
last=16000
size_of_itertrain=8000//nlists
size_of_itertest=8000//nlists
#print(size_of_itertrain)
for  i in range(nlists+1):
      
      end=(i+1)*size_of_itertrain
      if(last<end):
        end=last
      xytrain=[x_train[i*size_of_itertrain:end],y_train[i*size_of_itertrain:end]]
      tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
      tdataset = EnglishTrainDataset(xytrain)
      tsampler=RandomSampler(tdataset)
      tdataloader = DataLoader(tdataset, batch_size=32, num_workers=1, shuffle=False,sampler=tsampler)
      
      epochs=2
      total_steps=len(tdataloader)*epochs
      sch=get_linear_schedule_with_warmup(optimizer,
                                    num_warmup_steps=0,num_training_steps=total_steps)

      end=(i+1)*size_of_itertest
      if(last<end):
        end=last 
    
      xytest=[x_test[i*size_of_itertest:end],y_test[i*size_of_itertest:end]]
      tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
      tedataset = EnglishTestDataset(xytest)  
      tesampler=RandomSampler(tedataset)
      tedataloader = DataLoader(tedataset, batch_size=32, num_workers=1, shuffle=False,sampler=tesampler)

      trainData(tdataloader,tedataloader)

      
y_test_pytorch=torch.tensor(xytest[1][index])'''

"nlists=2\nlast=16000\nsize_of_itertrain=8000//nlists\nsize_of_itertest=8000//nlists\n#print(size_of_itertrain)\nfor  i in range(nlists+1):\n      \n      end=(i+1)*size_of_itertrain\n      if(last<end):\n        end=last\n      xytrain=[x_train[i*size_of_itertrain:end],y_train[i*size_of_itertrain:end]]\n      tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)\n      tdataset = EnglishTrainDataset(xytrain)\n      tsampler=RandomSampler(tdataset)\n      tdataloader = DataLoader(tdataset, batch_size=32, num_workers=1, shuffle=False,sampler=tsampler)\n      \n      epochs=2\n      total_steps=len(tdataloader)*epochs\n      sch=get_linear_schedule_with_warmup(optimizer,\n                                    num_warmup_steps=0,num_training_steps=total_steps)\n\n      end=(i+1)*size_of_itertest\n      if(last<end):\n        end=last \n    \n      xytest=[x_test[i*size_of_itertest:end],y_test[i*size_of_itertest:end]]\n      tokenizer=bertTokenizer.from_pretrained('

In [0]:
#y_test_pytorch=torch.tensor(xytest[1][40])

In [0]:
#len(xytest[1])

80

In [0]:
#len(xytest[0])

80

In [0]:
#len(xytrain[1])

80

In [0]:

MAXLENGTH

21

In [0]:

print("MAXLENGTH",MAXLENGTH)

MAXLENGTH 21
